### 1.2.2 レイヤとしてのクラス化と順伝播の実装

- Python版ではレイヤをclassで定義していたが，Juliaではclassの概念が存在しない．
  - モデルをコンストラクタで定義し，それにメソッドを作用させていくように考える．
- そのため，レイヤに持たせていたメソッド`forward()`と`backward()`，インスタンス変数`params`と`grads`を独立させる．
  - メソッドは`function`で定義し，インスタンス変数は`struct`に持たせる．
  - Juliaの多重ディスパッチの機構を利用し，同じ名前のメソッドでも型によって異なる処理を呼び出す．
    - レイヤを型として定義する．

In [1]:
# 抽象型の定義
abstract type Layer end

In [2]:
# Sigmoidレイヤの定義
mutable struct Sigmoid <: Layer
    params
    # コンストラクタメソッド
    Sigmoid() = new()
end

In [3]:
# Affineレイヤの定義
mutable struct Affine <: Layer
    params
    W
    b
    function Affine(W, b)
        params = [W, b]
        new(params)
    end
end

In [4]:
# Python版のSigmoidクラスのforward()
function forward(self::Sigmoid, x) # デフォルトの引数を指定することを考慮して，selfを最初に指定する
    return 1.0 ./ (1.0 .+ exp.(-x))
end

forward (generic function with 1 method)

In [5]:
# Python版のAffineクラスのforward()
function forward(self::Affine, x)
    W, b = self.params
    return x * W .+ reshape(b, 1, :) # bを1x4 Arrayにreshape
end

forward (generic function with 2 methods)

In [6]:
# TwoLayerNetの定義
mutable struct TwoLayerNet
    I # input_size
    H # hidden_size
    O # output_size
    W1
    b1
    W2
    b2
    layers
    
    # コンストラクタメソッド
    function TwoLayerNet(I, H, O)
        # 重みとバイアスの初期化
        W1 = randn(I, H)
        b1 = randn(H)
        W2 = randn(H, O)
        b2 = randn(O)
        
        # レイヤの生成
        layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        new(I, H, O, W1, b1, W2, b2, layers)
    end
end

In [7]:
# Python版のTwoLayerNetのpredict()
function predict(model::TwoLayerNet, x)
    for layer in model.layers
        x = forward(layer, x)
    end
    return x
end

predict (generic function with 1 method)

In [8]:
x = randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = predict(model, x)

10×3 Array{Float64,2}:
 2.24208  -0.0326778  2.33101
 2.15943  -0.112144   2.87679
 1.31978  -0.357306   2.78874
 1.53852  -0.336512   2.64645
 2.79394   0.103816   2.71332
 1.83168  -0.114299   2.08651
 2.28209  -0.0529507  2.47476
 2.04563  -0.149579   2.50183
 1.88454  -0.225718   2.6791
 1.80019  -0.0252452  1.85104